## <center>Парсинг сайта Archive of Our Own, фандом - Genshin Impact</center>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

Функция для страницы:

In [2]:
def ao3_parcing(url):
    r = requests.get(url)
    time.sleep(1)
    soup = bs(r.text, "html.parser")
    
    fanfics_name = soup.find_all('h4', class_='heading')
    resname = []
    for name in fanfics_name:
        resname.append(str(name.select("a[href^='/works/']")))
    resname = [i.split('>')[1].split('<')[0].replace('amp;', '') for i in resname if len(i) > 2]
        
    fanfics_pairing = soup.find_all('ul', class_="tags commas")
    respairs = []
    for elem in fanfics_pairing:
        respairs.append(str(elem.find_all('li', class_='relationships')))
    for i in range(0, len(respairs)):
        respairs[i] = respairs[i].split('<li class="relationships">')
        respairs[i].remove(respairs[i][0])
        for j in range(0, len(respairs[i])):
            respairs[i][j] = respairs[i][j].split('>')[1].split('<')[0]
            respairs[i][j] = respairs[i][j].replace('amp;', '')
            
    fanfics_raitings = soup.select('span[class*="rating-"]')
    resraitings = []
    for elem in fanfics_raitings:
        resraitings.append(str(elem.find('span', class_='text').text))
    
    return resname, respairs, resraitings

Прогон по страницам:

In [3]:
finalnames = []
finalpairs = []
finalraitings = []

for i in range(1, 102):
    base = "https://archiveofourown.org/tags/%E5%8E%9F%E7%A5%9E%20%7C%20Genshin%20Impact%20(Video%20Game)/works?page="
    print(i)
    url = base + str(i)
    tempnames, temppairs, tempraitings = ao3_parcing(url)
    for j in range(0, len(tempnames)):
        finalnames.append(tempnames[j])
        finalpairs.append(temppairs[j])
        finalraitings.append(tempraitings[j])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


Сборка датасета:

In [4]:
for i in range(0, len(finalpairs)):
    if len(finalpairs[i]) == 0:
        finalpairs[i].append('no pairing')
        finalnames[i] = [finalnames[i]]
        finalraitings[i] = [finalraitings[i]]
    else:
        finalnames[i] = [finalnames[i] for j in range(0, len(finalpairs[i]))]
        finalraitings[i] = [finalraitings[i] for j in range(0, len(finalpairs[i]))]
        

flatnames = [item for sublist in finalnames for item in sublist]
flatpairs = [item for sublist in finalpairs for item in sublist]
flatraitings = [item for sublist in finalraitings for item in sublist]
data = pd.DataFrame(list(zip(flatnames, flatraitings, flatpairs)), columns =['name', 'raiting', 'pairings'])
display(data.head(30))
print(data.info()) 

,name,raiting,pairings
0,Likewise | Diluc,General Audiences,Diluc (Genshin Impact)/Reader
1,Risky Business,Mature,Tartaglia | Childe/Ying | Lumine (Genshin Impact)
2,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo/Kaeya (Genshin Impact)
3,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo & Traveler (Genshin Impact)
4,"I’ve handled the wood, but I still worship the...",General Audiences,Diluc & Kaeya (Genshin Impact)
5,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo & Kaeya & Klee (Genshin Impact)
6,Earth Grows As Water Erodes,Mature,Tartaglia | Childe/Zhongli (Genshin Impact)
7,A Flower Without A Seed,Explicit,Baizhu (Genshin Impact) & Reader
8,A Flower Without A Seed,Explicit,Baizhu (Genshin Impact)/You
9,A Flower Without A Seed,Explicit,Baizhu (Genshin Impact) and OC (You)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      5307 non-null   object
 1   raiting   5307 non-null   object
 2   pairings  5307 non-null   object
dtypes: object(3)
memory usage: 124.5+ KB
None


In [5]:
#data.to_excel('raw-ao3-genshin.xlsx')

Удалим ненужное упоминание фандома:

In [8]:
data['pairings'] = data['pairings'].str.replace(' (Genshin Impact)', '', regex=False)
data = data.drop_duplicates()
display(data.head(30))
print(data.info()) 

,name,raiting,pairings
0,Likewise | Diluc,General Audiences,Diluc/Reader
1,Risky Business,Mature,Tartaglia | Childe/Ying | Lumine
2,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo/Kaeya
3,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo & Traveler
4,"I’ve handled the wood, but I still worship the...",General Audiences,Diluc & Kaeya
5,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo & Kaeya & Klee
6,Earth Grows As Water Erodes,Mature,Tartaglia | Childe/Zhongli
7,A Flower Without A Seed,Explicit,Baizhu & Reader
8,A Flower Without A Seed,Explicit,Baizhu/You
9,A Flower Without A Seed,Explicit,Baizhu and OC (You)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5302 entries, 0 to 5306
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      5302 non-null   object
 1   raiting   5302 non-null   object
 2   pairings  5302 non-null   object
dtypes: object(3)
memory usage: 165.7+ KB
None


In [12]:
print(data['pairings'].unique()[0:500])

['Diluc/Reader' 'Tartaglia | Childe/Ying | Lumine' 'Albedo/Kaeya'
 'Albedo & Traveler' 'Diluc & Kaeya' 'Albedo & Kaeya & Klee'
 'Tartaglia | Childe/Zhongli' 'Baizhu & Reader' 'Baizhu/You'
 'Baizhu and OC (You)' 'Baizhu & Original Female Character(s)'
 'Baizhu/Reader' 'Baizhu & You' 'Raiden Ei | Baal/Yae Miko'
 'Kong | Aether/Xiao | Alatus' 'Kamisato Ayaka/Ying | Lumine'
 'Kaedehara Kazuha/Scaramouche/Shikanoin Heizou'
 'Kaedehara Kazuha/Scaramouche' 'Kaedehara Kazuha/Shikanoin Heizou'
 'Scaramouche/Shikanoin Heizou' 'Past Childe/Scaramouche'
 'Kamisato Ayato/Thoma' 'Albedo/Venti' 'Arataki Itto/Gorou'
 'One-Sided Hu Tao/Ying | Lumine' 'Arlecchino/Columbina'
 'Arlecchino/Hu Tao' 'Albedo/Kong | Aether'
 'Kong | Aether/Tartaglia | Childe' 'Kong | Aether/Venti'
 'Kaeya/Kong | Aether' 'Kaedehara Kazuha/Kong | Aether'
 'Kamisato Ayato/Kong | Aether' 'Kong | Aether/Shikanoin Heizou'
 'Kong | Aether/Everyone' 'Kong | Aether/Zhongli' 'Kong | Aether/Thoma'
 'Diluc/Kong | Aether' 'Kong | Aether/Sc

Здесь - в отличие от фикбука - всё и так отсортировано в лексикографическом порядке, а уточнений не много - удалим их:

In [15]:
dupl_keywords = ['Past ', 'past ', 'one-sided ', 'One-sided ', 'one-sided', 'One-sided', 
                 '(one-sided)', '(One-sided)', ' - Relationship']

for keyword in dupl_keywords:
    data['pairings'] = data['pairings'].str.replace(keyword, '', regex=False)
data = data.drop_duplicates()
display(data.head(30))
print(data.info())

,name,raiting,pairings
0,Likewise | Diluc,General Audiences,Diluc/Reader
1,Risky Business,Mature,Tartaglia | Childe/Ying | Lumine
2,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo/Kaeya
3,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo & Traveler
4,"I’ve handled the wood, but I still worship the...",General Audiences,Diluc & Kaeya
5,"I’ve handled the wood, but I still worship the...",General Audiences,Albedo & Kaeya & Klee
6,Earth Grows As Water Erodes,Mature,Tartaglia | Childe/Zhongli
7,A Flower Without A Seed,Explicit,Baizhu & Reader
8,A Flower Without A Seed,Explicit,Baizhu/You
9,A Flower Without A Seed,Explicit,Baizhu and OC (You)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5302 entries, 0 to 5306
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      5302 non-null   object
 1   raiting   5302 non-null   object
 2   pairings  5302 non-null   object
dtypes: object(3)
memory usage: 165.7+ KB
None


In [17]:
#data.to_excel('ao3-genshin.xlsx')